In [35]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
import pandas as pd
from pandas import ExcelWriter
from collections import Counter
import math
def init(fileName):
    global ws_driver,pos_driver,ner_driver,result,filename

    filename = "./_target/"+fileName+".xls"
    result="./_target/"+fileName+"reslut.xls"
    file = pd.read_excel(filename)

    #//print(file.head(1))
    data_with_index = file.set_index("公開/公告號")
    for i in data_with_index.index:
        if('A' in i):
            data_with_index=data_with_index.drop(i)
    print(data_with_index.head(1))
    # * filter of the 公開專利 -> the 公開 one is pretty useless
    file =data_with_index
    print(len(file))
    return file

In [36]:
def run_file (file):
    var=file["摘要"]
    name=file["專利名稱"]
    global sort_key,sort_words,sent_cnt,document,TF_IDF
    sort_key=[]
    sort_words=[]
    for i in range (len(file)) :
        text =[var[i]]
        sent_cnt=0
        
        self_list=[]## get combine word 
        common_list=[]## get the most common word 
        document=[]
        # * Run pipeline
        # //print("Running pipeline ... WS")
        ws = ws_driver(text)
        # //print("Running pipeline ... POS")
        pos = pos_driver(ws)
        # // print("Running pipeline ... NER")
        
        ner = ner_driver(text)
        
        print("Running pipeline ... done")
        
        print()
        def pack_ws_pos_sentece(sentence_ws, sentence_pos):
            assert len(sentence_ws) == len(sentence_pos)
            res = []
                
            #TODO 字詞的組合 根據屬性表將名詞-> 專有名詞 來增加字詞的特殊性
            pre_pos=''
            pre_ws=''
            for word_ws, word_pos in zip(sentence_ws, sentence_pos):
                #! 寫的好爛... 可以透過語言結構做優化
                res.append(f"{word_ws}({word_pos})")
                
                if(word_pos=="Na" or "V" in word_pos):
                    common_list.append(word_ws)
                    
                if((word_pos == "Na"or word_pos=="VC") and pre_pos!='' ):
                # * use 
                    Nb=f"{pre_ws}{word_ws}"
                    # // print(Nb)
                    self_list.append(pre_ws+word_ws)
                    # //res.append(f"{Nb}(Nb)")
                    pre_pos=""
                    pre_ws=""
                        
                else :
                    #* memorizing
                    if(word_pos=="VC" or word_pos=="VD" or word_pos=="Na" or word_pos=="Nb"):
                        pre_pos=word_pos
                        pre_ws=word_ws
                        
                    else :
                        pre_pos=''
                        pre_ws=''
                        # //res.append(f"{word_ws}({word_pos})")
                if(word_pos=="PERIODCATEGORY" or word_pos=="SEMICOLONCATEGORY"):
                    sent_cnt+1
                    print(word_ws+""+str(sent_cnt))
                    
                    document.append(res)
                
            return "\u3000".join(res)
        
        
        # *zip指標的標記
        for sentence, sentence_ws, sentence_pos, sentence_ner in zip(text, ws, pos, ner):
            # //print(pos)
            pack_ws_pos_sentece(sentence_ws, sentence_pos)
        sort_key.append(Counter(self_list).most_common(20))

        sort_words.append(Counter(common_list).most_common(20))
        print(name[i])
        print("done")
        # //print(sentence)
        print("------------")
        TF_IDF=[]
        print(sort_key[i])
        print("------------")
        print(sort_words[i])
        
        temp=[]
        for words in sort_key[i]:
            # print(words[1])
            # print()
            _TF=((words[1])/len(pos)) 
            _IDF=math.log(words[1]/sent_cnt)
            print(_TF*_IDF)
            temp.append(words[0],float("{:.2f}".format(_TF*_IDF)))
        TF_IDF.append(temp)

        print(TF_IDF)

        # _TF_IDF=float("{:.2f}".format(_TF*_IDF))
        # *for entity in sentence_ner:
        #//     print(entity)
        print()
        del text
        del self_list,common_list,document 
    return 

In [37]:
def file_done(file):
    #* date format
    ori_time=file["申請日"]
    date=[]
    for i in ori_time:
        s=str(i)
        date.append(s[0:4]+'/'+s[4:6]+'/'+s[6:8])
        #//(datetime(year=int(), month=int(s[4:6]), day=int(s[6:8])))
    #//print((date))

    #* export back to the file 
    print(len(TF_IDF))

    bank ={
            # "ID":file.index,
            "申請日":date,
            "名稱":file["專利名稱"],
            "申請人":file["申請人"],
            "IPC":file["IPC"],
            "引用專利":file["引用專利"],
            "被參考次數":file["被參考次數"],    
            "專利摘要":file["摘要"],
            "摘要關鍵字":sort_key,
            "摘要常用字":sort_words,
            "TF_IDF":TF_IDF,
            }
    print (pd.DataFrame(bank))
    pd.DataFrame(bank).drop_duplicates(subset ="名稱",keep = False, inplace = True)
    pd.DataFrame(bank).to_excel(result)
    return 
#// bank ={"名稱":name,
#//         "KEY":sort_key,
#//         "WORD":sort_words
#//         }
#// print (pd.DataFrame(bank))

In [38]:
print("Initializing drivers ... WS")
ws_driver = CkipWordSegmenter(level=2)
print("Initializing drivers ... POS")
pos_driver = CkipPosTagger(level=2)
print("Initializing drivers ... NER")
ner_driver = CkipNerChunker(level=2)
print("Initializing drivers ... done")
print()
#for i in ["中信金控","兆豐金控","台新金控","台灣金融控股","國泰金控","第一金控","玉山金控","土地銀行","富邦金控","開發金"]:
for i in ["開發金"]:
    file=init(i)
    run_file(file)
    file_done(file)
    print(i+" is done")

Initializing drivers ... WS
Initializing drivers ... POS
Initializing drivers ... NER


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Initializing drivers ... done

              公開/公告日          申請號       申請日  \
公開/公告號                                        
TWM605596U  20201221  TW109209603  20200727   

                                                         專利名稱  \
公開/公告號                                                          
TWM605596U  用於遠距查看及調整金融商品交易策略之遠端控制系統Remote control system ...   

                                                 申請人  \
公開/公告號                                                 
TWM605596U  凱基證券股份有限公司; KGI SECURITIES CO. LTD. (TW)   

                                                           摘要  \
公開/公告號                                                          
TWM605596U  ﻿本創作揭露一種用於遠距查看及調整金融商品交易策略之遠端控制系統，其係裝設於行動裝置，行動裝...   

                            IPC  引用專利  被參考次數  
公開/公告號                                        
TWM605596U  G06Q 40/00(2012.01)   NaN    NaN  
6


Inference: 100%|██████████| 1/1 [00:00<00:00,  1.77it/s]

Running pipeline ... done

；0
；0
。0
。0
用於遠距查看及調整金融商品交易策略之遠端控制系統Remote control system for remote viewing and adjustment of financial commodity trading strategies
done
------------
[('控制系統', 4), ('通訊模組', 3), ('行動裝置', 2), ('交易系統', 2), ('金融商品', 2), ('交易策略', 2), ('創作揭露', 1), ('調整金融', 1), ('商品交易', 1), ('電腦主機', 1), ('控制電腦', 1), ('傳送指令', 1), ('使用者', 1), ('利用行動', 1)]
------------
[('系統', 6), ('交易', 5), ('控制', 5), ('係', 5), ('模組', 5), ('金融', 3), ('商品', 3), ('策略', 3), ('行動', 3), ('裝置', 3), ('電訊', 3), ('連接', 3), ('通訊', 3), ('查看', 2), ('電腦', 2), ('主機', 2), ('供以', 2), ('運行', 2), ('有', 2), ('創作', 1)]


ZeroDivisionError: division by zero